In [9]:
import pandas as pd
import numpy as np
import math
import random
import csv
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import get_linear_schedule_with_warmup


In [10]:
# some parameters
epochs = 5
learning_rate = 5e-5 #default 1e-3 5e-5
epsilon = 1e-8 #default
model_name = "gpt2"
batch_size = 4

# this produces sample output every 100 steps
sample_every = 100
# save the model every 1000 step
save_every = 1000
# save the model to this file name
save_model = "models/topsmallest_epochs"

In [17]:
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize, pos_tag
import re

# Ensure NLTK downloads
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')

stop_words = set(stopwords.words('english'))

def preprocess_ingredients(ingredients):
    ingredients_list = eval(ingredients)
    processed_ingredients = []
    regex = re.compile('[^a-zA-Z ]')
    lemmatizer = WordNetLemmatizer()
    #stemmer = PorterStemmer()
    
    # POS tags that represent nouns
    noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']
    
    # Define the words to be dropped
    #words_to_drop = {"powder", "brown", "salt", "water", "sugar", "onion", "butter", "pepper", "ground", "cream"} 

    for ingr in ingredients_list:
        ingr = regex.sub(' ', ingr.lower()).strip()
        components = [comp.strip() for comp in ingr.split('and')]
        

        for comp in components:
                        
            sentence = ""
            tokens = word_tokenize(comp)  # Tokenize each component
            tagged_tokens = pos_tag(tokens)  # Perform POS tagging
            
            # Extract main nouns while handling compound nouns
            nouns = []
            current_noun = ""
            for word, tag in tagged_tokens:
                word = lemmatizer.lemmatize(word.strip())
                if len(word) > 2 and word not in stop_words and tag in noun_tags: # and word not in words_to_drop
                    if current_noun:
                        nouns.append(current_noun)
                        current_noun = ""
                    current_noun = word
            
            # Add last current noun if exists
            if current_noun:
                nouns.append(current_noun)            
            
            for word in nouns:
                singular_comp = lemmatizer.lemmatize(word.strip())
                #stemmed_word = stemmer.stem(singular_comp)
            
                if singular_comp not in stop_words and len(singular_comp) > 2:
                    sentence += singular_comp + " "
                    
            if sentence.strip():
                processed_ingredients.append(sentence.strip())

    return list(set(processed_ingredients))

# Funzione di preprocessing per le tecniche
def preprocess_techniques(techniques):
    techniques_list = eval(techniques)
    processed_techniques = []

    for technique in techniques_list:
        technique = technique.lower().strip()
        tokens = word_tokenize(technique)
        tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
        processed_techniques.append(' '.join(tokens))

    return processed_techniques

def normalize_fractions(step):
    # Rimuove gli spazi attorno al simbolo della frazione
    return re.sub(r'(\d+)\s*/\s*(\d+)', r'\1/\2', step)

def preprocess_steps(steps):
    steps_list = eval(steps)
    processed_steps = []

    for step in steps_list:
        step = step.lower().strip()
        step = normalize_fractions(step)
        tokens = word_tokenize(step)
        tokens = [re.sub(r"[^a-zA-Z0-9,./-]+", "", token) for token in tokens]
        tokens = [token for token in tokens if token and token != '--']
        processed_steps.append(' '.join(tokens))

    return processed_steps

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matteorigat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/matteorigat/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/matteorigat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matteorigat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/matteorigat/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [18]:
# Funzione per preprocessare ogni riga della ricetta
def process_recipe(row):
    ingredients = preprocess_ingredients(row['ingredients'])
    techniques = preprocess_techniques(row['techniques_list'])
    instructions = preprocess_steps(row['steps'])
    #instructions = row['steps'].lower().replace('\'', '').replace('[', '').replace(']', '').replace('"', '')
    
    ingredients_str = ', '.join(ingredients)
    techniques_str = ', '.join(techniques)
    instructions_str = '. '.join(instructions)
    instructions_str = instructions_str.replace(" ,", ",")

    recipe_instance = f"[BOS] [INGREDIENTS] {ingredients_str} [TECHNIQUES] {techniques_str} [STEPS] {instructions_str} [EOS]"
    return recipe_instance

In [19]:
from joblib import Parallel, delayed
import multiprocessing

# Funzione per preprocessare i dati in parallelo
def load_preprocess_raw_data_parallel(raw_data):
    with open(raw_data, 'r', encoding='utf-8') as f:
        reader = list(csv.DictReader(f))  # Convertiamo reader in una lista
        num_cores = multiprocessing.cpu_count()  # Otteniamo il numero di core della CPU
        print(f"Number of CPU cores: {num_cores}")
        
        # Eseguiamo il preprocessing in parallelo
        recipe_instances = Parallel(n_jobs=num_cores)(
            delayed(process_recipe)(row) for row in tqdm(reader, desc="Processing recipes", unit="recipes")
        )

    return recipe_instances

In [20]:
# create text list for dataset
# https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions/data
#recipe_list = load_preprocess_raw_data("dataset/RAW_merged.csv")
recipe_list = load_preprocess_raw_data_parallel("dataset/RAW_merged_top_smallest.csv")
recipe_list = recipe_list[:100]

train_list, test_list = np.split(recipe_list, [int(.8 * len(recipe_list))])
print('\nNumber of train data: ', len(train_list))
print('Number of test data: ', len(test_list))

Number of CPU cores: 8


Processing recipes: 100%|██████████| 6010/6010 [00:01<00:00, 5643.26recipes/s]



Number of train data:  80
Number of test data:  20


In [43]:
print(train_list[25])

[BOS] [INGREDIENTS] cooking spray, sweet potato, salt, kosher salt, oil, sesame seed, oori paste, pepper flake [TECHNIQUES] bake, pour [STEPS] preheat oven to 375 degrees. line small, rimmed baking sheet with foil. spray with cooking spray. add cubed sweet potatoes, tandoori paste, sesame seeds, veg oil, 1/2 teaspoon salt, and red pepper flakes to bowl. stir until the sweet potatoes are well coated. pour onto the prepared baking sheet separate into a single layer. bake 20 minutes on bottom shelf. remove from oven with spatula turn over stir the sweet potatoes. place bake into the oven for additional 15-20 minutes. watch closely. they may need to be moved to top shelf. potatoes should begin to crisp and not be too dark. remove from oven. sprinkle with reserved 1/2 teaspoon of kosher salt [EOS]


In [8]:
# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='[BOS]', eos_token='[EOS]', pad_token='[PAD]')
# add special tokens for title, ingredients and instruction seperator
special_tokens_dict = {'additional_special_tokens': ['[INGREDIENTS]', '[TECHNIQUES]', '[STEPS]']} # '[INGR]', '[STEP]'
# check the number of special tokens
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print('We have added', num_added_toks, 'tokens')

We have added 3 tokens


In [9]:
# Verifica gli ID dei token speciali
special_tokens = ['[BOS]', '[EOS]', '[PAD]', '[INGREDIENTS]', '[TECHNIQUES]', '[STEPS]'] # '[INGR]'
for token in special_tokens:
    token_id = tokenizer.convert_tokens_to_ids(token)
    print(f"Token: {token}, ID: {token_id}")

Token: [BOS], ID: 50257
Token: [EOS], ID: 50258
Token: [PAD], ID: 50259
Token: [INGREDIENTS], ID: 50260
Token: [TECHNIQUES], ID: 50261
Token: [STEPS], ID: 50262


In [10]:
# Esempio di testi
texts = [
    train_list[0]
]

# Tokenizzazione
encodings = tokenizer.batch_encode_plus(
    texts,
    truncation=True,
    max_length=320,
    padding='max_length',
    return_tensors='pt',
    add_special_tokens=True  # Assicurati che i token speciali siano inclusi
)

# Visualizza gli input_ids e attention_mask
print("Input IDs:")
print(encodings['input_ids'])
print("\nAttention Mask:")
print(encodings['attention_mask'])

Input IDs:
tensor([[50257,   220, 50260,   537,   659,    11, 37792,    11, 13135,    11,
         21772,  3056,    11,  8268,    11,  9267, 24240,   220, 50261, 20667,
            11, 29901,    11,  1667,  4559,    11, 32857,    11, 12153,   220,
         50262,  5022,   477,  9391,  1978,   290,   900,   319,  3753, 26547,
         10491,    11,   262,  8268,   481,  2222,   503,   262, 13135,   286,
           262, 23972,    11,  6938,   290,   751,   517,  1622,   654,   284,
           534,  2614,  6938,    11,   345,   460,   779,  4713, 37792,   393,
         23751,  1030,    78,   611,  9871,   837,   655,  3505,   284, 20720,
         32566,   290,   779,   257,  1310,  1342,   621, 16577,    11,  3544,
          1058,  4691,   351,  2048,   597,  9799,   355,   257,  1735,    11,
           319,  1353,   286,   257,  4077, 20698,    11,   779,   355,   257,
         34366,   329,   865,   385,  2395, 25854,  1377,   655, 30506, 23972,
          2427,   286, 49289,    11,   77

In [11]:
# Decodifica dei token
decoded_inputs = [tokenizer.decode(ids, skip_special_tokens=False) for ids in encodings['input_ids']]
for i, decoded in enumerate(decoded_inputs):
    print(f"Decoded Input {i}: {decoded}")

Decoded Input 0: [BOS]  [INGREDIENTS]  clove, basil, juice, virgin oil, salt, rom tomato  [TECHNIQUES]  boil, grill, marinate, simmer, toss  [STEPS]  mix all ingredients together and set on counter stirring occasionally, the salt will bring out the juice of the tomatoes, taste and add more seasonings to your personal taste, you can use fresh basil or oregano if preferred, just remember to chopped finely and use a little less than dried, uses : serve with almost any meal as a side, on top of a green salad, use as a topping for bruschetta -- just chop tomatoes instead of slicing, use for caprese salad -- add sliced buffalo mozzarella and you're done, have some leftover juice : place some sliced chicken breasts and marinate over night, then carefully remove the chicken and quickly saut them until done, add remaining juice and tomatoes, boil for 1-2 minutes and then add cooked penne pasta, toss, and simmer for 5-6 minutes, juice uses : you can reuse the `` marinade '' with new tomatoes, ju

In [12]:
lengths = [len(tokenizer.encode(recipe)) for recipe in recipe_list]
max_length_in_data = max(lengths)
avg_length_in_data = sum(lengths) / len(lengths)
print(f"Lunghezza massima: {max_length_in_data}, Lunghezza media: {avg_length_in_data}")

# Lunghezza massima: 312, Lunghezza media: 136.59567387687187

Lunghezza massima: 294, Lunghezza media: 138.17


In [13]:
class GPT2Dataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length=320):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = [text.lower() for text in txt_list]  # Preprocess texts to lowercase
        
        # Tokenize all texts in a batch
        encodings = tokenizer.batch_encode_plus(
            self.texts,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors='pt',
            add_special_tokens=True
        )
        
        self.input_ids = encodings['input_ids']
        self.attn_masks = encodings['attention_mask']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [14]:
dataset = GPT2Dataset(train_list, tokenizer, max_length=320)

# Split into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

   64 training samples
   16 validation samples


In [15]:
# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
    train_dataset,  # The training samples.
    sampler=RandomSampler(train_dataset),  # Select batches randomly
    batch_size=batch_size  # Trains with this batch size.
)

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
    val_dataset,  # The validation samples.
    sampler=SequentialSampler(val_dataset),  # Pull out batches sequentially.
    batch_size=batch_size  # Evaluate with this batch size.
)

In [16]:
# Carica il modello
# I'm not really doing anything with the config buheret
pretrained = False
try:
    configuration = GPT2Config.from_pretrained(model_name, output_hidden_states=False)
    model = GPT2LMHeadModel.from_pretrained(save_model, config=configuration, ignore_mismatched_sizes=True)
    tokenizer = GPT2Tokenizer.from_pretrained(save_model)
    
    checkpoint = torch.load(f"{save_model}_checkpoint.pth")
    pretrained = True
except:
    configuration = GPT2Config.from_pretrained(model_name, output_hidden_states=False)
    model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)
    

model.resize_token_embeddings(len(tokenizer))

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=learning_rate,
                              eps=epsilon
                              )
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs
print('Total number of steps: ', total_steps)

warmup_steps = int(0.05 * total_steps) # 5% del totale degli step
# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)
start_epoch = 0
loss = 9999999999

if(pretrained):
    # Ripristina il modello, l'ottimizzatore e lo scheduler
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    # Imposta l'epoca da cui riprendere
    start_epoch = checkpoint['epoch'] + 1
    loss = checkpoint['loss']
    print(f"Resuming training from epoch {start_epoch}")


# Tell pytorch to run this model on the GPU.
if torch.cuda.is_available():
    device = torch.device("cuda")
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
else:
    device = torch.device("cpu")

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if device == "cuda":
    torch.cuda.manual_seed_all(seed_val)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at models/topsmallest_epochs and are newly initialized because the shapes did not match:
- transformer.wte.weight: found shape torch.Size([50263, 768]) in the checkpoint and torch.Size([50257, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Total number of steps:  80
Resuming training from epoch 5


In [17]:
training_stats = []
print("Currently using device type: ", device)

model = model.to(device)

for epoch_i in range(start_epoch, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    losses = []

    total_train_loss = 0

    model.train()

    loop = tqdm(train_dataloader, leave=True)
    for step, batch in enumerate(loop):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(b_input_ids,
                        labels=b_labels,
                        attention_mask=b_masks
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss
        losses.append(batch_loss)

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:
            print('Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.'.format(step, len(train_dataloader), batch_loss))

        loss.backward()

        optimizer.step()

        scheduler.step()

        if step % save_every == 0:
            model.save_pretrained(save_model)
            torch.save({
                'epoch': epoch_i,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': loss,
            }, f"{save_model}_checkpoint.pth")

        loop.set_postfix(loss=batch_loss)

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Calculate perplexity.
    losses = torch.tensor(losses)
    train_perplexity = math.exp(torch.mean(losses))

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Perplexity: {0:.2f}".format(train_perplexity))
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    model.eval()

    losses = []
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids,
                            attention_mask=b_masks,
                            labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        losses.append(batch_loss)
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Calculate perplexity.
    losses = torch.tensor(losses)
    val_perplexity = math.exp(torch.mean(losses))

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation perplexity: {0:.2f}".format(val_perplexity))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Perplexity': train_perplexity,
            'Valid. Perplexity': val_perplexity,
        }
    )
    
    break

print("")
print("Training complete!")

Currently using device type:  cpu

Training complete!


In [18]:
print(epoch_i)
print(loss)

NameError: name 'epoch_i' is not defined

In [281]:
# Salva il modello
if(start_epoch < epochs):
    model.save_pretrained(save_model)
    tokenizer.save_pretrained(save_model)
    
    # Salva lo stato dell'ottimizzatore e dello scheduler
    torch.save({
        'epoch': epoch_i,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'loss': loss,
    }, f"{save_model}_checkpoint.pth")
else:
    print("Modello già salvato")

Modello già salvato
